In [1]:
from mesmerize_core import *
import tifffile
import numpy as np
from matplotlib import pyplot as plt

# Paths
These are the only variables you will need to modify in the demo notebook accoridng to your own `caiman_data` dir path

In [2]:
# set the parent directory as the top-level directory for your experiment data
# this is always mandatory

# for the demo set this as the path to the `caiman_data` dir
set_parent_raw_data_path("/home/kushal/caiman_data/")

# this path can be anywhere, algorithm output files will be organized within here
batch_path = "/home/kushal/caiman_data/mesmerize-core-batch/batch.pickle"

# path to raw movie tiff file for the demo, must be within `raw data path` or `batch path`
movie_path = "/home/kushal/caiman_data/example_movies/Sue_2x_3000_40_-46.tif"

In [3]:
# create a new batch
df = create_batch(batch_path)
df

,algo,name,input_movie_path,params,outputs,comments,uuid


# Motion correction

In [4]:
# params, exactly the same as what you'd directly use with CaImAn
mcorr_params1 =\
{
  'main': # this key is necessary for specifying that these are the "main" params for the algorithm
    {
        'max_shifts': [24, 24],
        'strides': [48, 48],
        'overlaps': [24, 24],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

# Add a "batch item", this is the combination of:
* algorithm to run, `algo`
* input movie to run the algorithm on, `input_movie_path`
* parameters for the specified algorithm, `params`
* a name for you to keep track of things, usually the same as the movie filename, `name`

In [5]:
# add an item to the batch
df.caiman.add_item(
  algo='mcorr',
  name='my_movie',
  input_movie_path=movie_path,
  params=mcorr_params1
)
df

,algo,name,input_movie_path,params,outputs,comments,uuid
0,mcorr,my_movie,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': [24, 24], 'strides': [...",None,None,02e7d053-ce7b-41b1-9240-2236321a08e3


We can now see that there is one item in the batch dataframe, we can add another item with the same input movie but with different parameters

In [6]:
# We create another set of params, useful for gridsearches for example
mcorr_params2 =\
{
  'main':
    {
        'max_shifts': [24, 24],
        'strides': [24, 24],
        'overlaps': [12, 12],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

# add other param variant to the batch
df.caiman.add_item(
  algo='mcorr',
  name='my_movie',
  input_movie_path=movie_path,
  params=mcorr_params2
)
df

,algo,name,input_movie_path,params,outputs,comments,uuid
0,mcorr,my_movie,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': [24, 24], 'strides': [...",None,None,02e7d053-ce7b-41b1-9240-2236321a08e3
1,mcorr,my_movie,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': [24, 24], 'strides': [...",None,None,815fb4fe-f6c6-4477-9b41-970a3f0d24e5


# Run batch item(s)
You can run single batch items by calling `caiman.run()` on a `Series` (row) of the DataFrame. You can also use a `for` loop to run multiple batch items

In [7]:
# run the first "batch item"
# this will run in a subprocess by default
process = df.iloc[0].caiman.run()
process.wait()

starting mc


100%|██████████| 1/1 [00:00<00:00, 12.15it/s]

100%|██████████| 1/1 [00:00<00:00,  9.85it/s]


mc finished successfully!
computing projections
Computing correlation image
finished computing correlation image


0

Reload the DataFrame to see the outputs information for the mcorr batch item

In [8]:
df = load_batch(batch_path)
# we can see that the output for the first item is not empty
df

,algo,name,input_movie_path,params,outputs,comments,uuid
0,mcorr,my_movie,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': [24, 24], 'strides': [...",{'mean-projection-path': 02e7d053-ce7b-41b1-92...,None,02e7d053-ce7b-41b1-9240-2236321a08e3
1,mcorr,my_movie,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': [24, 24], 'strides': [...",None,None,815fb4fe-f6c6-4477-9b41-970a3f0d24e5


Check if the algorithm ran successfully 

In [9]:
# True if the algo ran succesfully
df.iloc[0]["outputs"]["success"]

True

In [10]:
# use extensions to get the full path the outputs and other useful things
# you should always use the extensions
# never handle the "outputs" dict manually!
df.iloc[0].mcorr.get_output_path()

PosixPath('/home/kushal/caiman_data/mesmerize-core-batch/02e7d053-ce7b-41b1-9240-2236321a08e3/02e7d053-ce7b-41b1-9240-2236321a08e3-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000_.mmap')

In [11]:
# similarly resolve the full input movie path
df.iloc[0].caiman.get_input_movie_path()

PosixPath('/home/kushal/caiman_data/example_movies/Sue_2x_3000_40_-46.tif')

Get the input movie and mcorr so we can visualize them

In [12]:
# Load the input and output movies so we can view them
input_movie = tifffile.imread(df.iloc[0].caiman.get_input_movie_path())

# extension for loading the mcorr movie
mcorr_movie = df.iloc[0].mcorr.get_output()

# Visualization using fastplotlib
You must have `fastplotlib` installed to visualize the mcorr and raw movie

In [13]:
from fastplotlib import GridPlot, Image, Plot
from ipywidgets.widgets import IntSlider, VBox

# Raw & MCorr movie side-by-side
This cell just plots the raw & mcorr movie side by side and uses a slider for updating the frame index for random-access scrolling

In [14]:
# gridplot with 1 row, 2 columns
gp = GridPlot(
    shape=(1, 2),
    controllers=np.array([[0, 0]]) # this is to that we can pan-zoom in sync :D 
)

input_graphic = Image(
    data=input_movie[0], # image data for the first frame
    cmap='gnuplot2' # my favorite colormap
)

mcorr_graphic = Image(
    data=mcorr_movie[0], # image data for the first frame
    cmap='gnuplot2'
)

# add the grahpics to the subplots
gp.subplots[0, 0].add_graphic(input_graphic)
gp.subplots[0, 1].add_graphic(mcorr_graphic)

# make a GUI slider
slider = IntSlider(value=0, min=0, max=input_movie.shape[0] - 1, step=1)

previous_value = 0

# a function to update the frame based on the slider value
def update_frame():
    global previous_value
    if previous_value == slider.value:
        return
    
    input_graphic.update_data(input_movie[slider.value])
    mcorr_graphic.update_data(mcorr_movie[slider.value])
    
# add to the animation so that this is run in every render cycle
gp.add_animations([update_frame])

# show the gridplot and slider
VBox([gp.show(), slider])

RFBOutputContext()

**Center the scenes**

In [15]:
gp.subplots[0, 0].center_scene()

**Close the canvas if your don't have a powerful GPU so that you can plot more visualizations later.**

In [16]:
gp.canvas.close()

# CNMF

We can continue from mcorr above and perform CNMF using the mcorr output

In [17]:
# some params for CNMF
params_cnmf =\
{
    'main': # indicates that these are the "main" params for the CNMF algo
        {
            'p': 1,
            'gnb': 2,
            # raises error: no parameter 'merge_thresh' found
            'merge_thr': 0.85,
            'rf': 15,
            'stride_cnmf': 6,
            'K': 4,
            'gSig': [4, 4],
            'ssub': 1,
            'tsub': 1,
            'method_init': 'greedy_roi',
            'min_SNR': 2.0,
            'rval_thr': 0.7,
            'use_cnn': True,
            'min_cnn_thr': 0.8,
            'cnn_lowest': 0.1,
            'decay_time': 0.4,
        },
    'refit': True, # If `True`, run a second iteration of CNMF
}

# add a batch item
df.caiman.add_item(
  algo='cnmf',
  name='my_movie',
  input_movie_path=df.iloc[0].mcorr.get_output_path(),  # use mcorr output from a previous item
  params=params_cnmf
)

# run this item
process = df.iloc[-1].caiman.run()
process.wait()

cnmf params {'main': {'p': 1, 'gnb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride_cnmf': 6, 'K': 4, 'gSig': [4, 4], 'ssub': 1, 'tsub': 1, 'method_init': 'greedy_roi', 'min_SNR': 2.0, 'rval_thr': 0.7, 'use_cnn': True, 'min_cnn_thr': 0.8, 'cnn_lowest': 0.1, 'decay_time': 0.4}, 'refit': True}


making memmap


2022-08-03 20:38:05.255177: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-03 20:38:05.255209: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: hantman-calcium
2022-08-03 20:38:05.255214: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: hantman-calcium
2022-08-03 20:38:05.255325: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.129.6
2022-08-03 20:38:05.255344: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.129.6
2022-08-03 20:38:05.255349: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.129.6
2022-08-03 20:38:05.255563: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU

performing CNMF
fitting images
refitting
Eval
USING MODEL:/home/kushal/caiman_data/model/cnn_model.json
5/5 [==============================] - 0s 9ms/step


0

### Look at the CNMF outputs

In [18]:
df = load_batch(batch_path)
df

,algo,name,input_movie_path,params,outputs,comments,uuid
0,mcorr,my_movie,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': [24, 24], 'strides': [...",{'mean-projection-path': 02e7d053-ce7b-41b1-92...,None,02e7d053-ce7b-41b1-9240-2236321a08e3
1,mcorr,my_movie,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': [24, 24], 'strides': [...",None,None,815fb4fe-f6c6-4477-9b41-970a3f0d24e5
2,cnmf,my_movie,02e7d053-ce7b-41b1-9240-2236321a08e3/02e7d053-...,"{'main': {'p': 1, 'gnb': 2, 'merge_thr': 0.85,...",{'mean-projection-path': 0c3f9d3b-828e-486b-81...,None,0c3f9d3b-828e-486b-81f2-e3c70bb5aa93


In [19]:
df.iloc[-1]

algo                                                             cnmf
name                                                         my_movie
input_movie_path    02e7d053-ce7b-41b1-9240-2236321a08e3/02e7d053-...
params              {'main': {'p': 1, 'gnb': 2, 'merge_thr': 0.85,...
outputs             {'mean-projection-path': 0c3f9d3b-828e-486b-81...
comments                                                         None
uuid                             0c3f9d3b-828e-486b-81f2-e3c70bb5aa93
Name: 2, dtype: object

In [20]:
df.iloc[-1]["outputs"]["success"]

True

# Load outputs
For visualization

In [21]:
# get the motion corrected movie as a memmap
cnmf_movie = df.iloc[-1].cnmf.get_input_memmap()

# we can get the contours of the spatial components
coors, coms = df.iloc[-1].cnmf.get_contours(component_indices="good")

# get bad components
coors_bad, coms_bad = df.iloc[-1].cnmf.get_contours(component_indices="bad")

# Plot contours for good components on the motion corrected movie

In [22]:
# create a simple plot
plot = Plot()

# plot the first frame, transpose so it lines up with contours
image_graphic = plot.image(data=cnmf_movie[0].T, cmap='gray')

# plot all the contours
for coor in coors:
    # line data has to be 3D
    zs = np.ones(coor.shape[0]) # this will place it above the image
    coors_3d = np.dstack([coor[:, 0], coor[:, 1], zs])[0].astype(np.float32)
    
    # red color, just [R, G, B, A] -> red, green, blue, alpha (transparency)
    colors = np.vstack([[1., 0., 0., 1.]] * coors_3d.shape[0]).astype(np.float32)
    plot.line(data=coors_3d, colors=colors)

# make a slider like for mcorr
slider_cnmf = IntSlider(value=0, min=0, max=cnmf_movie.shape[0] - 1, step=1)
    
previous_ix = 0

# animate func just like mcorr viz
def update_frame_cnmf():
    if slider_cnmf.value == previous_ix:
        return
    # update with frame from slider value, again transpose to line up with contours
    image_graphic.update_data(data=cnmf_movie[slider_cnmf.value].T)

plot.add_animations([update_frame_cnmf])
    
VBox([plot.show(), slider_cnmf])

RFBOutputContext()

In [23]:
plot.canvas.close()

# View the reconstructed movie, residuals, and reconstructed background

In [24]:
# gridplot with 1 row, 2 columns
gp_rcm = GridPlot(
    shape=(2, 2),
    controllers="sync"
)

# for the mcorr movie
left_graphic = Image(
    data=cnmf_movie[0].T, # image data for the first frame
    cmap='gnuplot2' # my favorite colormap
)

# get the first frame of reconstructed movie
rcm0 = df.iloc[-1].cnmf.get_rcm(component_indices="good", frame_indices=0)[0]

# create the graphic
right_graphic = Image(
    data=rcm0.T, # image data for the first frame
    vmin=0,
    vmax=100,
    cmap='gnuplot2'
)

# residuals
resid0 = df.iloc[-1].cnmf.get_residuals(frame_indices=0)[0]
resid_graphic = Image(
    data=resid0,
    cmap="gnuplot2",
)

# reconstructed background, b * f
recon_bg0 = df.iloc[-1].cnmf.get_rcb(frame_indices=0)[0]
recon_bg_graphic = Image(
    data=recon_bg0,
    cmap="gnuplot2",
)


# add the grahpics to the subplots
gp_rcm.subplots[0, 0].add_graphic(left_graphic)
gp_rcm.subplots[0, 1].add_graphic(right_graphic)
gp_rcm.subplots[1, 0].add_graphic(resid_graphic)
gp_rcm.subplots[1, 1].add_graphic(recon_bg_graphic)

# make a GUI slider
slider_rcm = IntSlider(value=0, min=0, max=cnmf_movie.shape[0] - 1, step=1)

previous_value = 0

# a function to update the frame based on the slider value
def update_rcm_frame():
    global previous_value
    if previous_value == slider_rcm.value:
        return
    
    left_graphic.update_data(cnmf_movie[slider_rcm.value].T)
        
    # get the A x C for just 1 frame
    rcm_frame = df.iloc[-1].cnmf.get_rcm(component_indices="good", frame_indices=slider_rcm.value)[0]
    right_graphic.update_data(rcm_frame.T)
    
    # residuals for 1 frame
    resid_frame = df.iloc[-1].cnmf.get_residuals(frame_indices=slider_rcm.value)[0]
    resid_graphic.update_data(resid_frame)
    
    # b * f for 1 frame
    recon_bg_frame = df.iloc[-1].cnmf.get_rcb(frame_indices=slider_rcm.value)[0]
    recon_bg_graphic.update_data(recon_bg_frame)
    
    previous_value = slider_rcm.value
    
# add to the animation so that this is run in every render cycle
gp_rcm.add_animations([update_rcm_frame])
VBox([gp_rcm.show(), slider_rcm])

RFBOutputContext()

In [25]:
gp_rcm.subplots[0,0].center_scene()